In [18]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

seg_df = pd.read_csv('http://bit.ly/PMR-ch5')



In [21]:
#Create test data for observations to look at frequency and chisquare
tmp = np.repeat(range(4),[25,25,25,20])
tmp

#Use nunique to get count of each value
tmp_values, tmp_counts = np.unique(tmp,return_counts = True)
tmp_counts

#Import scipy and stats
from scipy import stats
stats.chisquare(tmp_counts)
#Shows 85% chance of seeing a dataset with differences similar to ours

#changing observations to see a different number
tmp_values, tmp_counts = np.unique(np.repeat(range(4),[25,25,25,10]),
                                   return_counts = True)
print(tmp_counts)
print(stats.chisquare(tmp_counts))
print('Expected values: {}'.format(np.ones(4)*tmp_counts.sum()/4))

#Same data but 1/5 to see difference
tmp_counts_small = tmp_counts / 5
print(tmp_counts_small)
print(stats.chisquare(tmp_counts_small))
print('Expected values: {}'.format(np.ones(4)*tmp_counts_small.sum()/4))

#Looking at high differences
tmp_counts_large = tmp_counts * 10 + 1900
print(tmp_counts_large)
print(stats.chisquare(tmp_counts_large))
print('Expected values: {}'.format(np.ones(4)*tmp_counts_large.sum()/4))


#Looking at our segment data and chisquare - are they sig different? - Yes
segment_values, segment_counts = np.unique(seg_df.Segment,
                                           return_counts = True)
print(segment_counts)
stats.chisquare(segment_counts)

#Using the value_counts() to look at segment counts
seg_df.Segment.value_counts()

#value_counts can be passed directly to chisquare
stats.chisquare(seg_df.Segment.value_counts())

[25 25 25 10]
Power_divergenceResult(statistic=7.9411764705882355, pvalue=0.047243183430928606)
Expected values: [21.25 21.25 21.25 21.25]
[5. 5. 5. 2.]
Power_divergenceResult(statistic=1.5882352941176472, pvalue=0.6620603202525777)
Expected values: [4.25 4.25 4.25 4.25]
[2150 2150 2150 2000]
Power_divergenceResult(statistic=7.988165680473372, pvalue=0.04625691960442831)
Expected values: [2112.5 2112.5 2112.5 2112.5]
[ 70 100  80  50]


Power_divergenceResult(statistic=17.333333333333336, pvalue=0.0006034948344733137)